# SQL Sentiment Analysis with SQLite + OpenAI

This lab mirrors the Snowflake Cortex sentiment demo using SQLite for storage and OpenAI for scoring.

**Learning objectives**
- Store reviews in SQLite
- Analyze sentiment with an LLM
- Aggregate results using SQL


In [ ]:
# Optional: install deps if running in a fresh environment
# !pip install openai python-dotenv


In [ ]:
import os
import sqlite3
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))


In [ ]:
conn = sqlite3.connect('reviews.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS reviews (
        id INTEGER PRIMARY KEY,
        review_text TEXT,
        sentiment_score REAL,
        sentiment_label TEXT
    )
''')

reviews = [
    'The product quality is amazing, I absolutely loved it!',
    'Terrible experience. Nothing worked as expected.',
    'It was okay, not great, not terrible.',
    'The packaging was damaged but customer support helped quickly.'
]


In [ ]:
def analyze_sentiment(text):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {
                'role': 'system',
                'content': 'Rate the sentiment from -1 (very negative) to 1 (very positive). Return only a number.'
            },
            {'role': 'user', 'content': text}
        ]
    )
    score = float(response.choices[0].message.content.strip())
    label = 'Positive' if score > 0.3 else 'Negative' if score < -0.3 else 'Neutral'
    return score, label


In [ ]:
cursor.execute('DELETE FROM reviews')
for i, review in enumerate(reviews, start=1):
    score, label = analyze_sentiment(review)
    cursor.execute(
        'INSERT INTO reviews (id, review_text, sentiment_score, sentiment_label) VALUES (?, ?, ?, ?)',
        (i, review, score, label)
    )
conn.commit()


In [ ]:
# Average sentiment
cursor.execute('SELECT AVG(sentiment_score) as avg_sentiment FROM reviews')
cursor.fetchone()


In [ ]:
# Group by sentiment
cursor.execute('''
    SELECT sentiment_label, COUNT(*) as count
    FROM reviews
    GROUP BY sentiment_label
''')
cursor.fetchall()


In [ ]:
# Most negative reviews
cursor.execute('''
    SELECT review_text, sentiment_score
    FROM reviews
    WHERE sentiment_label = 'Negative'
    ORDER BY sentiment_score ASC
''')
cursor.fetchall()


## Snowflake Cortex mapping

| Snowflake Cortex | SQLite + OpenAI equivalent |
|---|---|
| `SNOWFLAKE.CORTEX.SENTIMENT(text)` | `client.chat.completions.create(...)` + prompt |
